# Large Test Data Generator

### Author: Vivek Maswadkar

import all the required libraries<br/>
**random** is to generate random numbers and strings<br/>
**phonenumbers** is a library that gives us sample phone numbers for specified country<br/>
**sring** and **time** are inbuilt libraries provided by python

In [ ]:
import random
import json
import string
import time
import pymongo
import uuid
import os

In [ ]:
uri = uri = 'mongodb://' + os.environ.get('MONGOUSER') +  ':' + os.environ.get('MONGOPASSWORD') + '@' + os.environ.get('MONGOSERVER') + ':27017'

Initial f_country_array from weights and country codes 

In [ ]:
def initializeCountryList():
    country_client = pymongo.MongoClient(uri)
    country_db = country_client.iso
    country_coll = country_db.details
    country_cursor = country_coll.find({'for_address':1})

    countryList = []
    for a in country_cursor:
        temp=[]
        temp.append(a['alpha-2'])
        temp=temp*int(a['weight'])
        countryList = countryList+temp
    return countryList

Initialize phone number list

In [ ]:
def initializePhoneList():
        phone_client = pymongo.MongoClient(uri)
        phone_db = phone_client.iso
        phone_coll = phone_db.details
        phone_cursor = phone_coll.find({},{'_id':0,'alpha-2':1,'dialCode':1,'eg_phone_number':1})
        
        phoneList=[]
        
        for phoneNumber in phone_cursor:
            phoneList.append(phoneNumber)
        
        return phoneList
        

function which gives you a random phone number for the given country code<br/>
**country code** is ISO ALPHA-2 Code e.g. switzerland code is **CH**

In [ ]:
def get_phonenumber(region_code):
    global f_phone_array
    for phone in f_phone_array:
        if phone['alpha-2'] == region_code:
            #example_number = phonenumbers.example_number(region_code)
            phone_number_length = len(str(phone['eg_phone_number']))
            random_phone_number = str(random.randrange(10**int(phone_number_length-1),10**int(phone_number_length)))
            return("+" + str(phone['dialCode']) + random_phone_number)
            break

this function gives you a random date between **start** date and **end** date given<br/>
**prop** is the random number between 0 and 1<br/>
**format** is the format mentioned in time.strftime<br/>

In [ ]:
def randomDateBetween(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)
    
    return time.strftime(format, time.localtime(ptime))

is valid card

In [ ]:
def is_valid_card(cardnumber):
    def digits_of(n):
        return [int(d) for d in str(n)]

    digits = digits_of(cardnumber)
    #print(digits)
    odd_numbers= digits[-1::-2]
    even_numbers = digits[-2::-2]
    total_sum= sum(odd_numbers)
    #print(total_sum)
    for x in even_numbers:
        total_sum += (sum(digits_of(x*2)))
    #print(total_sum)
    return total_sum % 10 == 0

Credit card number generator

In [ ]:
def getCreditCard(country,bank_name,card_type):
    client = pymongo.MongoClient(uri)
    db = client.creditcards
    cursor = db.details
    
    x = cursor.find({'country':country,'bank_name':bank_name,'card_type':card_type})
    this_instance = x[random.randint(0,x.count()-1)]
    prefix = this_instance['bin_range']
    
    length = int(this_instance['number_length'])
    creditCard = prefix
    for i in range(1,length-len(prefix)):
        creditCard+=str(random.choice(range(0,10)))
    
    for i in range(0,10):
        pp = creditCard+str(i)
        if is_valid_card(pp):
            creditCard = pp
            break
    
    return creditCard

address from mongoDB

In [ ]:
def getAddress(country):
    address_client = pymongo.MongoClient(uri)
    address_db = address_client.postal_address
    address_cursor = address_db.details
    
    x = address_cursor.aggregate([{'$sample':{'size':1}},{'$match':{'country':country}},{ '$out' : 'random_address' }])
    address_client.close()
    
    
    address_client = pymongo.MongoClient(uri)
    address_db = address_client.postal_address
    address_cursor = address_db.random_address
    
    if address_cursor.count() == 0:
        return None
    else:
        singleone = address_cursor.find({},{'_id':0})
        return singleone[0]

In [ ]:
def getSampleAddress(country):
    retry_count=0
    while retry_count< 5:
        singleAddress =  getAddress('CH')
        if singleAddress != None:
            retry_count = 99
            return singleAddress
        else:
            retry_count+=1

unique values from the list

In [ ]:
def initializeList(filename):
    f = open(filename,encoding='utf8')
    templist = []
    for i in f:
        templist.append(i.rstrip('\n'))
    templist = list(set(templist))
    return templist

In [ ]:
def getitemfromlist(filename):
    global myfile
    if filename in myfile:
        if len(myfile[filename]) > 0:
            return myfile[filename].pop(random.randint(0,len(myfile[filename])-1))
        else:
            return None
    else:
        filearray = initializeList(filename)
        myfile[filename] = filearray
        return myfile[filename].pop(random.randint(0,len(myfile[filename])-1))

In [ ]:
def getanyitemfromlist(filename):
    global myfile
    if filename in myfile:
        if len(myfile[filename]) > 0:
            return myfile[filename][random.randint(0,len(myfile[filename])-1)]
        else:
            return None
    else:
        filearray = initializeList(filename)
        myfile[filename] = filearray
        return myfile[filename][random.randint(0,len(myfile[filename])-1)]

In [ ]:
def initializeDB(parameter):
    global data
    address_client = pymongo.MongoClient(uri)
    address_db = address_client.postal_address
    address_cursor = address_db.details
    
    x = address_cursor.aggregate([{'$sample':{'size':10000}},{'$match':{'country':parameter}},{ '$out' : 'random_address' }])
    address_client.close()
    
    
    address_client = pymongo.MongoClient(uri)
    address_db = address_client.postal_address
    address_cursor = address_db.random_address
    
    addresses = address_cursor.find({},{'_id':0})
    
    templist = []
    for each_address in addresses:
        templist.append(each_address)
    
    return templist

In [ ]:
def getitemfromDB(parameter):
    global myfile
    if parameter in myfile:
        if len(myfile[parameter]) > 0:
            return myfile[parameter].pop(0)
        else:
            return None
    else:
        myfile[parameter] = initializeDB(parameter)
        return myfile[parameter].pop(0)

In [ ]:
def initializecreditcardList(country, bank, cardtype):
    cc_client = pymongo.MongoClient(uri)
    cc_db = cc_client.creditcards
    cc_cursor = cc_db.details
    
    cc = cc_cursor.find({'country':country,'bank_name':bank,'card_type':cardtype},{'_id':0})
    
    templist = []
    for each_cc in cc:
        templist.append(each_cc)
    
    return templist

In [ ]:
def db_getCreditCard(country, bank, cardtype):
    global myfile
    parameter = country + bank + cardtype
    if parameter in myfile:
        if len(myfile[parameter]) > 0:
            x=myfile[parameter]
        else:
            return None
    else:
        myfile[parameter] = initializecreditcardList(country, bank, cardtype)
        x=myfile[parameter]


    this_instance = random.choice(x)
    prefix = this_instance['bin_range']
    
    length = int(this_instance['number_length'])
    creditCard = prefix
    for i in range(1,length-len(prefix)):
        creditCard+=str(random.choice(range(0,10)))
    
    for i in range(0,10):
        pp = creditCard+str(i)
        if is_valid_card(pp):
            creditCard = pp
            break
    
    return creditCard


<br/>This is the core function of the program<br>
this creates one row at a time depending on column definations mentioned in our 
<span style="font-family:courier">
"master_parameters.json" </span> file<br>

In [ ]:
def create_row(column_definations):
    row_value = ""
    global f_country_array
    #f_country_array = ["AU","CA","DE","ES","GB"] + ["CH"]*96        
    f_country = random.choice(f_country_array)  
    
    for x in column_definations:
        
        #each if block if for datatype. You can add your own datatypes
        #e.g. datatype == "email" can be string.ascii_letters + string.digits + '@'
        if x["datatype"] == "string":
            if x["is_variable_legth"]:
                if x["is_null"]:
                    ll = random.randint(0,x["length"])
                else:
                    ll = random.randint(1,x["length"])
            else:
                ll = x["length"]
            value_of_string = ''.join(random.choice(string.ascii_letters + string.digits + 'äëöü') for _ in range(ll))
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "file":
            
            value_of_string = getanyitemfromlist(x["file_path"])
            
            #f= open(x["file_path"],encoding="utf8")
            #test = []
            #for row in f:
            #    test.append(row.rstrip("\n"))
            #value_of_string=random.choice(test)
            value_of_string = '"' + value_of_string + '"'
        ####################
        
        elif x["datatype"] == "ssn":
            value_of_string = str(random.randrange(100,999)) + "-" + str(random.randrange(10,99)) + "-" + str(random.randrange(100,999))
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "number":
            value_of_string = str(random.randrange(10**int(x["min_range"]),10**int(x["max_range"])))
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "phonenumber":
            value_of_string = get_phonenumber(f_country)
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "xdate":
            value_of_string = randomDateBetween(x["from_date"], x["until_date"],x["date_format"], random.random())
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "country":
            value_of_string = f_country
            value_of_string = '"' + value_of_string + '"'
            
        ####################
        elif x["datatype"] == "address":
            f_address= open("input/" + f_country + ".csv",encoding="utf8")
            address_array = []
            for row in f_address:
                address_array.append(row.rstrip("\n"))
            value_of_string=random.choice(address_array)
            value_of_string = '"' + value_of_string + '"'
        ####################        
        elif x["datatype"] == "creditcard":
            value_of_string = db_getCreditCard(x['country'],x['bank_name'],x['card_type'])
            value_of_string = '"' + value_of_string + '"'
        ####################
        elif x["datatype"] == "mongo_address":
            value_of_string = getitemfromDB(f_country)
            value_of_string = '"' + str(value_of_string) + '"'
        ####################        
        elif x["datatype"] == "unique_values":
            value_of_string = getitemfromlist(x['file_path'])
            value_of_string = '"' + str(value_of_string) + '"'
        ####################        
        elif x["datatype"] == "mychoice":
            value_of_string = random.choice(x['choices'])
            value_of_string = '"' + str(value_of_string) + '"'
        ####################        
        elif x["datatype"] == "uuid":
            value_of_string = uuid.uuid4()
            value_of_string = '"' + str(value_of_string) + '"'
        ####################        

        
        
        if row_value == "":
            row_value = value_of_string
        else:
            row_value = row_value + separator+ value_of_string
    return(row_value)

This is the **main** function which 
 - iterates for number of rows mentioned
 - writes the rows in the file

In [ ]:
%%time

f_country_array=initializeCountryList()
f_phone_array = initializePhoneList()
myfile = {}


#read the parameter file and parse it for further processing
with open('customer_master_parameters.json') as data_file:
    data = json.load(data_file)
separator=(data["separator"])


f = open(data["filename"],'w',encoding="utf8")
for i in range(data["number_of_rows"]):
    each_row=create_row(data["columns"]).rstrip('\n')
    print(each_row ,file=f)
f.close()

In [ ]:
myfile.keys()

## END